In [19]:
import pandas as pd
from ydata_profiling import ProfileReport

In [24]:
df = pd.read_csv('csv/uspevaemost.csv', encoding='cp1251')

df = df[df['Квалификация'] == 'Бакалавр']

In [21]:
# df['Студент'].nunique()

In [22]:
df = df[(df['Студент'] == '42477A384446554E566E55587779514855326A5674413D3D') & (df['Учебный год'] == '2011/2012')]
# df.sort_values(by='Учебный год')
# df.to_csv('test1.csv', index= False)

In [23]:
df = df[~(df['Оценка'] == 'неудовлетворительно') | ~(df.duplicated(subset=df.columns.difference(['Оценка']))) ]

df['Оценка'] = df['Оценка'].map({
    'отлично': 5,
    'хорошо': 4,
    'удовлетворительно': 3,
    'зачтено': 2, 
    'незачет': 1 
})

df.drop(columns=['Группа', "Дисциплина", "Специальность", "Форма обучения", "Квалификация", "Статус"], inplace=True)

df = df.groupby(['Студент', 'Семестр', 'Учебный год'], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)


df.head()

Оценка                                                                2  3
Студент                                          Семестр Учебный год      
42477A384446554E566E55587779514855326A5674413D3D 1       2011/2012    5  4
                                                 2       2011/2012    4  4

In [41]:
(df.groupby(['Студент']).agg({'Группа': 'nunique'})["Группа"] > 1).sum()
df = df.groupby(['Студент']).agg({'Группа': 'nunique'})
df[df['Группа'] != 1]

,Группа
Студент,


In [25]:
df['sem_start'] = df['Учебный год'].str[:4]
df['sem_end'] = df['Учебный год'].str[5:]

df_years = df.groupby(['Студент']).agg({'sem_start': 'min', 'sem_end': 'max'})

df_kol_let = (df_years['sem_end']).astype('int16') - (df_years['sem_start']).astype('int16')
print(df_kol_let.shape)
(df_kol_let <= 4).sum()

(2714,)


2009

In [26]:
df = df.iloc[:, :2]
df.drop_duplicates(inplace=True)
df.groupby(['Студент']).agg({'Группа': 'count'})["Группа"] > 1

Студент
2B2F3765476B77595568664E4B614C4F6A4E4B7167413D3D    True
2B2F397249583175764B6A74722B373239642F7053413D3D    True
2B2F594F58715179766342613567424C58306B4776513D3D    True
2B3242586235356D525A677A493141454B34667249413D3D    True
2B3272765979562B38442B4363364B4A396B527475413D3D    True
                                                    ... 
7A7850434F2B6B727242494A44694F6B4B79563143413D3D    True
7A79364932442B444567536D6A752B5446324C5853513D3D    True
7A7A38456B32345073425950566354506C2F4E3949773D3D    True
7A7A54716A794E562B66507A50467073345555397A413D3D    True
7A7A634B583179437670457A6569626A356B65626D513D3D    True
Name: Группа, Length: 4800, dtype: bool

In [3]:
df = df[df['Статус'] == 'Отчисленный']

df = df[df['Квалификация'] != 'Слушатель программ довузовской подготовки']

In [4]:
df.drop_duplicates(inplace=True)

df = df[~(df['Оценка'] == 'неудовлетворительно') | ~(df.duplicated(subset=df.columns.difference(['Оценка']))) ]

df['Оценка'] = df['Оценка'].map({
    'отлично': 5,
    'хорошо': 4,
    'удовлетворительно': 3,
    'зачтено': 2, 
    'незачет': 1 
})

df_bez_D = df.drop_duplicates(subset=df.columns.difference(['Оценка']), keep=False)

df_s_D = df.merge(df_bez_D, indicator=True, how='outer').loc[lambda x: x['_merge'] == 'left_only'].drop(columns=['_merge']).drop_duplicates()

In [5]:
max_indices = df_s_D.groupby(['Студент', 'Группа', 'Дисциплина', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'])['Оценка'].idxmax(skipna=True)
df_max = df_s_D.loc[max_indices]


min_indices = df_s_D.groupby(['Студент', 'Группа', 'Дисциплина', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'])['Оценка'].idxmin(skipna=True)
df_min = df_s_D.loc[min_indices]


In [6]:
df_min = df_min.groupby(['Студент', 'Группа', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)
df_max = df_max.groupby(['Студент', 'Группа', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)
df_bez_D = df_bez_D.groupby(['Студент', 'Группа', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)

In [7]:

df = pd.merge(df_bez_D, df_max, on=['Студент','Группа','Семестр','Учебный год','Специальность','Форма обучения','Квалификация','Статус'], how='outer')
df = pd.merge(df, df_min, on=['Студент','Группа','Семестр','Учебный год','Специальность','Форма обучения','Квалификация','Статус'], how='outer')

df.fillna(0, inplace=True)


In [8]:
df.rename(columns={'1_x': 'незачет', '2_x': 'зачет', '3_x': 'удовлетворительно', '4_x': 'хорошо', '5_x': 'отлично', '2_y': 'зачет испр', '3_y': 'удовлетворительно испр', '4_y': 'хорошо испр', '5_y': 'отлично испр', '1_y': 'незачет до испр', 2: 'зачет до испр', 3: 'удовлетворительно до испр', 4: 'хорошо до испр'}, inplace=True)

In [9]:

phrases = ['незачет', 'зачет', 'удовлетворительно', 'хорошо',
         'отлично', 'зачет испр', 'удовлетворительно испр',
         'хорошо испр', 'отлично испр', 'незачет до испр',
         'зачет до испр', 'удовлетворительно до испр', 'хорошо до испр']

df = df.sort_values(by=['Студент', 'Семестр'])
for phrase in phrases:
    df[f'Накоп {phrase}'] = df.groupby('Студент')[f'{phrase}'].cumsum()


In [10]:
df['незачет'] = df['незачет'].apply(lambda x: 1 if x >= 1 else 0)

df['отчислен'] = df.groupby('Студент')['незачет'].transform('max')

In [11]:
df = df[df['незачет'] < 1]
df = df[df['Накоп незачет'] < 1]



In [12]:
df.to_csv('tempData.csv', index=True)
df = pd.read_csv('tempData.csv')

df['Квалификация'] = df['Квалификация'].map({
    'Аспирант': 3,
    'Магистр': 2,
    'Специалист': 1,
    'Бакалавр': 0,
})

df['Форма обучения'] = df['Форма обучения'].map({
    'Заочная': 2,
    'Очная': 0,
   'Очно-заочная': 1
})

In [13]:

df.drop('Статус', axis=1, inplace=True)
df.drop('Студент', axis=1, inplace=True)
df.drop('Группа', axis=1, inplace=True)
df.drop('Специальность', axis=1, inplace=True)
df.drop('Учебный год', axis=1, inplace=True)
# df.drop('незачет', axis=1, inplace=True)
# df.drop('Накоп незачет', axis=1, inplace=True)


In [14]:
df.to_csv('csv/dataset.csv', index=False)




In [15]:
df.head()

,Семестр,Форма обучения,Квалификация,незачет,зачет,удовлетворительно,хорошо,отлично,зачет испр,удовлетворительно испр,...,Накоп отлично,Накоп зачет испр,Накоп удовлетворительно испр,Накоп хорошо испр,Накоп отлично испр,Накоп незачет до испр,Накоп зачет до испр,Накоп удовлетворительно до испр,Накоп хорошо до испр,отчислен
0,1,2,0,0,6.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,2,0,0,5.0,1.0,0.0,2.0,0.0,0.0,...,3.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0
2,3,2,0,0,2.0,0.0,2.0,1.0,0.0,0.0,...,4.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0
3,4,2,0,0,4.0,0.0,0.0,2.0,0.0,0.0,...,6.0,0.0,0.0,2.0,1.0,0.0,0.0,3.0,0.0,0
4,5,2,0,0,5.0,0.0,2.0,0.0,0.0,0.0,...,6.0,0.0,0.0,3.0,1.0,0.0,0.0,4.0,0.0,0


In [16]:
profile = ProfileReport(df, title="Profiling Report")
profile.to_file("your_reportXD1.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 10.15it/s]


df.to_csv('csv/uspevaemost1000.csv', index=False)